Import libraries

In [2]:
import pandas
import pickle
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import arviz
import pystan
import random

Get the model

In [42]:
# whether to recompile the stan program
DO_COMPILE = True

# stan program path
model_path = 'multi_feature.stan'

def build_model(path, pkl_file=None, do_compile=True):
    if do_compile:
        sm = pystan.StanModel(file=path)
        if pkl_file is not None:
            with open(pkl_file, 'wb') as f:
                pickle.dump(sm, f)

    # if the program hasn't been complied, check that the file already exists
    else: 
        if os.path.isfile(pkl_file):
            sm = pickle.load(open(pkl_file, 'rb'))
        else:
            raise FileNotFoundError
    return sm

sm = build_model(path = model_path, pkl_file='model.pkl', do_compile=DO_COMPILE)

Generate data 

In [18]:
# Simulate the data
num_samples = 600
sequence_length = 6
num_features = 3
exemplar_idx = np.repeat(np.arange(1, sequence_length+1), num_samples/sequence_length)

random.seed(1)

exemplar_means = np.random.randint(10, size = (sequence_length, num_features))

sig = np.identity(num_features);

sim_data = [np.random.multivariate_normal(exemplar_means[idx-1], sig) for idx in exemplar_idx] 
sim_data = np.asmatrix(sim_data)


Model parameters

In [12]:
# mu
mu_mean = 0.5
mu_sd = 1

# sd
sigma_alpha = 1
sigma_beta = 1

# noise SD
noise = 0.5

# environmental EIG
EIG_env = 1

# initialize data dict:
data = {"F": num_features, "mu_mean":mu_mean , "mu_sd": mu_sd, \
"sigma_alpha": sigma_alpha, "sigma_beta": sigma_beta, "noise": noise}


In [65]:
sample = True

# Initialize data
samples_from_current_stim = 1
total_samples = 1
exemplar_num = 1
model_LT = np.zeros((1, sequence_length))

sample_data = np.empty((num_samples,num_features))
sample_data[:] = np.nan

exemplar_labels = np.empty((num_samples,))
exemplar_labels[:] = np.nan

data["M"] = total_samples

# exemplar number 
data["K"] = exemplar_num

# add sim data
sample_data[total_samples-1] = sim_data[exemplar_idx == exemplar_num][samples_from_current_stim-1]
data["z"] = np.transpose(sample_data[0:total_samples,:])

# add exemplar for each id
exemplar_labels[total_samples-1] = int(exemplar_num)
data["exemplar_idx"] = [int(x) for x in exemplar_labels[~np.isnan(exemplar_labels)]]

data

{'F': 3,
 'mu_mean': 0.5,
 'mu_sd': 1,
 'sigma_alpha': 1,
 'sigma_beta': 1,
 'noise': 0.5,
 'M': 1,
 'K': 1,
 'Z': array([0.68054803, 9.11660374, 6.51538509]),
 'exemplar_idx': [1],
 'z': array([[0.34814784],
        [8.63719603],
        [3.48951179]])}

Action loop

In [ ]:
sample = True

# Initialize data
samples_from_current_stim = 1
total_samples = 1
exemplar_num = 1
model_LT = np.zeros((1, sequence_length))

sample_data = np.empty((num_samples,num_features))
sample_data[:] = np.nan

exemplar_labels = np.empty((1,num_samples))
exemplar_labels[:] = np.nan


while sample:
    
    # sample number
    data["M"] = total_samples

    # exemplar number 
    data["K"] = exemplar_num

    # add sim data
    sample_data[total_samples-1] = sim_data[exemplar_idx == exemplar_num,:][samples_from_current_stim-1]
    data["Z"] = sample_data[~np.isnan(sample_data)]

    # add exemplar for each id
    exemplar_labels[total_samples-1] = exemplar_num
    data["exemplar_idx"] = exemplar_labels[~np.isnan(exemplar_labels)]

    # get posterior samples

    # generate hypothetical next obs


    # get posterior predictive for each next obs
    post_preds = 1

    # compute KL for each hypothetical next obs
        # compute_KL(post, pre)

    fit = sm.sampling(data=data, iter=10000, chains=1)

    # get EIG
    EIG_stim = KLs * post_preds

    # decision rule
    if EIG_stim < EIG_env:
        sample = False
        model_LT[exemplar_num-1] = samples_from_current_stim

        # reset/increment counters
        samples_from_current_stim = 1
        exemplar_num =+ 1
    else:
        samples_from_current_stim =+ 1 
    
    total_samples =+1



RuntimeError: Exception: variable does not exist; processing stage=data initialization; variable name=z; base type=matrix_d  (in 'multi_feature.stan' at line 5)
